<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Paths" data-toc-modified-id="Paths-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Paths</a></span></li><li><span><a href="#Prepare-sqlite-database" data-toc-modified-id="Prepare-sqlite-database-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepare sqlite database</a></span></li><li><span><a href="#Iterate-through-crimes-dataset-and-add-to-SQL-database" data-toc-modified-id="Iterate-through-crimes-dataset-and-add-to-SQL-database-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Iterate through crimes dataset and add to SQL database</a></span></li></ul></div>

**Description**: Reads in raw crimes data contained in a .csv file and puts it
into a SQL database.

---

In [1]:
import sys
from datetime import datetime
from pathlib import Path

import pandas as pd

sys.path.append('../..')
from src.prepare_data.crime_database import get_engine

# Paths

In [ ]:
data_path = Path('../../data')
crimescsv_path = data_path / 'raw/Crimes_-_2001_to_present.csv'
sqldb_path = data_path / 'processed/crimes.db'

# Prepare sqlite database

If database already exists delete it

In [ ]:
if sqldb_path.is_file():
    sqldb_path.unlink()
    assert not sqldb_path.is_file()

Initialize engine to database
(actual database will be created when adding data to it)

In [ ]:
disk_engine = create_engine('sqlite:///' + str(sqldb_path))

# Iterate through crimes dataset and add to SQL database

Get "approximate" length of crimes dataset without actually loading
the datafile into memory. Length may be approximate as
if a .csv file contains quoted "\n" etc. which will result in
an overestimation of the dataset length. This is not a problem
as we only use the length for a convenient print statement.
The - 1 subtracts the header row.

In [ ]:
with crimescsv_path.open() as f:
    number_rows = sum(1 for line in f) - 1

In [ ]:
chunksize = 100000
start_time = datetime.now()
nrows_processed = 0
for i, df in enumerate(
        pd.read_csv(crimescsv_path, chunksize=chunksize, iterator=True)):
    # Parse dates
    df['Date'] = pd.to_datetime(df['Date'], format="%m/%d/%Y %I:%M:%S %p")
    # Only keep relevant columns
    df.drop(
        ['Case Number', 'X Coordinate', 'Y Coordinate'],
        axis='columns',
        inplace=True)

    df.to_sql('crimes', disk_engine, if_exists='append', index=False)

    nrows_processed += len(df)
    print(
        '{} seconds passed: completed {:,}/{:,} rows'.format(
            (datetime.now() - start_time).seconds, nrows_processed,
            number_rows),
        end='\r',
        flush=True)

print("Total time: {} seconds".format(
    (datetime.now() - start_time).seconds))
print("Rows processed: {:,}/{:,}".format(nrows_processed, number_rows))